# Practical Neighbourhood Connection to the Grid

#### Importing the needed packages
The first step in every python notebook is to install the needed packages (e.g. [`pandas`](https://pandas.pydata.org/docs/user_guide/index.html) for data management) as well as functions and classes from external scripts.

In [ ]:
from illuminator.engine import Simulation
import functions_T3
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

#### Setting
A neighbourhood of 10 houses is connected to the grid (see the picture below). This connection to the grid has a capacity of 15 kW.

![T3_basecase](Graphics_T3/neighbourhood_T3_base_case.png)

#### Task 1: Load on Connection in case of Conventional Energy Supply

In this task there is no local production of electricity in the neighbourhood. Therefore,  neighbourhood's demand needs to be completely fulfilled by electricity from the grid. Run the code below and observe the load at the connection for the two selected days (2012-02-01 and 2012-07-06). 

In [ ]:
# Scenario Data 
load_df = pd.read_csv('../Scenarios/load_data.txt', delimiter=',', header = 0,skiprows=1) # Load Profile
load_df['time'] = pd.to_datetime(load_df['time'])
load_df.set_index('time', inplace=True)

day_summer = '2012-07-06'
day_winter = '2012-02-01'

number_of_houses = 10 # size of the neighbourhood
connection_cap = 15 # capacity of the connection
tolerance_limit = 0.67 # a load on the connection between 67% and 90% is tolerable but causes a warning
critical_limit = 0.9 # a load on the connection over 90% of the connection capacity is critical

# Plot the Load on the connection
functions_T3.plot_load_on_connection(load_df, day_summer, number_of_houses, tolerance_limit, critical_limit, connection_cap)
functions_T3.plot_load_on_connection(load_df, day_winter, number_of_houses, tolerance_limit, critical_limit, connection_cap)

#### Task 2: Local Renewable Generation and Storage Solutions

To be more sustainable the neighbourhood decides to install solar panels on the roofs and a small-scale wind turbine. Additionally, the neighbourhood utilizes a communal battery to shift the consumption of the renewable energy according to the demand.

![T3_REScase](Graphics_T3/neighbourhood_T3_RES_case.png)


##### 2a) Integrating local Renewable Generation

Run the simulation below for the two chosen days. What do you observe and how does it compare to the case with conventional energy supply (task 1)? How does the installation of local renewables in the neighbourhood affect the load at the connection to the grid? Explain your observations.

In [ ]:
Components_RES_case = pd.Series(['Wind', 'PV', 'Load', 'Grid_Connection'])

#Input Data Simulation
number_houses = 10    #number of houses in the neighbourhood

pv_inputs = {'Module_area': 1.26, 'installed_capacity': 20000, 'Power_output_at_STC': 250} # the unit for power of the pv inputs is W

wind_inputs = {'p_rated': 10 , 'cp':0.40} # the unit for power of the wind inputs is kW

grid_inputs = {'Connection_Capacity': 15, 'Tolerance Limit': 0.67, 'Critical Limit': 0.9}

An overload at the connection to the grid can lead to damage of components, infrastructure and connected devices. Additionally, the reliability of the grid decreases if the load at the connection is frequently exceeding the capacity and blackouts can occur. Are overloads a problem in the case at hand? And what could be options to avoid overloads?

##### 2b) Local Renewables & Local Battery Storage
To utilize the local renewable generation more efficiently the neighbourhood decides to invest in a local community battery storage. Run the code below to simulate the new situation for the two chosen days.

In [ ]:
number_houses = 10    #number of houses in the neighbourhood

pv_inputs = {'Module_area': 1.26, 'installed_capacity': 20000, 'Power_output_at_STC': 250} # the unit for power of the pv inputs is W

wind_inputs = {'p_rated': 10, 'cp':0.40} # the unit for power of the wind inputs is kW


battery_inputs = {'max_p': 40, 'min_p': -40, 'max_energy': 40, 'initial_soc': 0, 'charge_efficiency': 0.9, 'discharge_efficiency': 0.9, 'soc_min': 10}

grid_inputs = {'Connection_Capacity': 15, 'Tolerance Limit': 0.67, 'Critical Limit': 0.9}

##### 2c) Bonus
Why do you think we included a summer and a winter day in each simulation step?

#### Task 3: Electrification of Assets

As part of the Energy Transition the neighbourhood has decided to disconnect from the gas network and install Air-Source Heat Pumps for each house for space heating and hot water. Furthermore, the households have switched to electric vehicles which they frequently charge at home. This has implications for the load on the connection of the neighbourhood to the grid. 

![T3_Elec_case](Graphics_T3/neighbourhood_T3_RES_Bat_Elec_case.png)

a) Examine the effect of electrifying assets by simulating the new situation with the code below. Compare this new situation to the previous cases. Make sure to simulate at least the given summer day and winter day for the case of electrified assets.

b) What advice would you give based on your observations to the residents that want to have a reliable electricity and heat supply while being sustainable.

### Task 4: Control Mechanism for Electrified Assets

The electrified assets namely the heat pump and the EV are assumed to be flexible loads in this exercise. This entails that the consumption does not necessarily need to happen at a certain point in time but rather within a time frame. To avoid overloading the connection to the grid each house implements a controller for their electrified assets.

a) Run the code below to simulate the new simulation for the summer day and the winter day. Compare the results with task 3. Feel free to also simulate other days. What impact does the control mechanism have?

In [ ]:
%%capture
CONFIG_FILE = 'Tutorial_physical_congestion_RES_bat_elec_assets.yaml'
simulation_RES_Bat = Simulation(CONFIG_FILE)
simulation_RES_Bat.set_model_param(model_name='CSVload', parameter='file_path', value='data/load_data.txt')
simulation_RES_Bat.set_model_param(model_name='CSV_pv', parameter='file_path',
                                   value='data/pv_data_Rotterdam_NL-15min.txt')
simulation_RES_Bat.set_model_param(model_name='CSV_wind', parameter='file_path', value='data/winddata_NL.txt')

new_settings = {'Wind1': {'p_rated': 0.3},
                'Load1': {'houses': 5},
                'PV1': {'cap': 500},
                'Battery1': {'max_p': 0.8, 'min_p': -0.8, 'soc_min': 10, 'soc_max': 90},
                'Grid1':{'connection_capacity' : 15, 'tolerance Limit': 0.67, 'critical Limit': 0.9},
                }

simulation_RES_Bat.edit_models(new_settings)

simulation_RES_Bat.set_scenario_param('start_time', '2012-06-01 00:00:00')
simulation_RES_Bat.set_scenario_param('end_time', '2012-06-01 23:45:00')

# run the simulation
simulation_RES_Bat.run()

b) How do you suspect the control mechanism works?